## **Código tomado de la documentación de giotto-tda**

In [ ]:
import numpy as np
from pathlib import Path


def make_point_clouds(n_samples_per_shape: int, n_points: int, noise: float):
    """Make point clouds for circles, spheres, and tori with random noise.
    """
    circle_point_clouds = [
        np.asarray(
            [
                [np.sin(t) + noise * (np.random.rand(1)[0] - 0.5), np.cos(t) + noise * (np.random.rand(1)[0] - 0.5), 0]
                for t in range((n_points ** 2))
            ]
        )
        for kk in range(n_samples_per_shape)
    ]
    # label circles with 0
    circle_labels = np.zeros(n_samples_per_shape)

    sphere_point_clouds = [
        np.asarray(
            [
                [
                    np.cos(s) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
                    np.cos(s) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                    np.sin(s) + noise * (np.random.rand(1)[0] - 0.5),
                ]
                for t in range(n_points)
                for s in range(n_points)
            ]
        )
        for kk in range(n_samples_per_shape)
    ]
    # label spheres with 1
    sphere_labels = np.ones(n_samples_per_shape)

    torus_point_clouds = [
        np.asarray(
            [
                [
                    (2 + np.cos(s)) * np.cos(t) + noise * (np.random.rand(1)[0] - 0.5),
                    (2 + np.cos(s)) * np.sin(t) + noise * (np.random.rand(1)[0] - 0.5),
                    np.sin(s) + noise * (np.random.rand(1)[0] - 0.5),
                ]
                for t in range(n_points)
                for s in range(n_points)
            ]
        )
        for kk in range(n_samples_per_shape)
    ]
    # label tori with 2
    torus_labels = 2 * np.ones(n_samples_per_shape)

    point_clouds = np.concatenate((circle_point_clouds, sphere_point_clouds, torus_point_clouds))
    labels = np.concatenate((circle_labels, sphere_labels, torus_labels))

    return point_clouds, labels


def make_gravitational_waves(
    path_to_data: Path,
    n_signals: int = 30,
    downsample_factor: int = 2,
    r_min: float = 0.075,
    r_max: float = 0.65,
    n_snr_values: int = 10,
        ):
    def padrand(V, n, kr):
        cut = np.random.randint(n)
        rand1 = np.random.randn(cut)
        rand2 = np.random.randn(n - cut)
        out = np.concatenate((rand1 * kr, V, rand2 * kr))
        return out

    Rcoef = np.linspace(r_min, r_max, n_snr_values)
    Npad = 500  # number of padding points on either side of the vector
    gw = np.load(path_to_data / "gravitational_wave_signals.npy")
    Norig = len(gw["data"][0])
    Ndat = len(gw["signal_present"])
    N = int(Norig / downsample_factor)

    ncoeff = []
    Rcoeflist = []

    for j in range(n_signals):
        ncoeff.append(10 ** (-19) * (1 / Rcoef[j % n_snr_values]))
        Rcoeflist.append(Rcoef[j % n_snr_values])

    noisy_signals = []
    gw_signals = []
    k = 0
    labels = np.zeros(n_signals)

    for j in range(n_signals):
        signal = gw["data"][j % Ndat][range(0, Norig, downsample_factor)]
        sigp = int((np.random.randn() < 0))
        noise = ncoeff[j] * np.random.randn(N)
        labels[j] = sigp
        if sigp == 1:
            rawsig = padrand(signal + noise, Npad, ncoeff[j])
            if k == 0:
                k = 1
        else:
            rawsig = padrand(noise, Npad, ncoeff[j])
        noisy_signals.append(rawsig.copy())
        gw_signals.append(signal)

    return noisy_signals, gw_signals, labels

In [ ]:
#from data.generate_datasets import make_point_clouds
n_samples_per_class = 10
point_clouds, labels = make_point_clouds(n_samples_per_class, 10, 0.1)
point_clouds.shape
print(f"There are {point_clouds.shape[0]} point clouds in {point_clouds.shape[2]} dimensions, "
      f"each with {point_clouds.shape[1]} points.")

There are 30 point clouds in 3 dimensions, each with 100 points.


In [ ]:
from gtda.homology import VietorisRipsPersistence

VR = VietorisRipsPersistence(homology_dimensions=[0, 1, 2])
diagrams = VR.fit_transform(point_clouds)
diagrams.shape

(30, 174, 3)

In [ ]:
pip install -U giotto-tda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.4/526.4 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.0 MB/s eta 0:00:00


In [ ]:
from gtda.plotting import plot_diagram

plot_diagram(diagrams[0])

In [ ]:
plot_diagram(diagrams[1])

In [ ]:
plot_diagram(diagrams[5])

In [ ]:
plot_diagram(diagrams[15])

In [ ]:
plot_diagram(diagrams[17])

In [ ]:
plot_diagram(diagrams[25])

In [ ]:
plot_diagram(diagrams[29])

# **Distancia Cuello de Botella**

In [ ]:
from gtda.diagrams import PairwiseDistance

In [ ]:
dist1 = PairwiseDistance(metric = 'bottleneck')
d1 = dist1.fit(X=[diagrams[0]])
d2 = dist1.transform([diagrams[1]])
print(d2)

[[0.03224662]]


In [ ]:
dist2 = PairwiseDistance(metric = 'bottleneck')
d4 = dist2.fit(X = diagrams[0:3])
d5 = dist2.transform(diagrams[3:6])
print(d5)

[[0.02983826 0.02918523 0.03349067]
 [0.03062851 0.03475664 0.03925739]
 [0.0323929  0.04158326 0.03475823]]


In [ ]:
dist3 = PairwiseDistance(metric = 'bottleneck')
d6 = dist3.fit(X = [diagrams[0]])
d7 = dist3.transform([diagrams[15]])
print(d7)

[[0.86455665]]


In [ ]:
dist4 = PairwiseDistance(metric = 'bottleneck')
d8 = dist4.fit(X = diagrams[0:10])
d9 = dist4.transform(diagrams[20:30])
print(d9)

[[0.78845779 0.78707248 0.78575735 0.7888063  0.79791817 0.78775338
  0.79238036 0.78613761 0.79111384 0.7847126 ]
 [0.79787684 0.79179535 0.79967906 0.7951661  0.80227877 0.79825452
  0.79943536 0.80328017 0.78407174 0.78981717]
 [0.78500486 0.78203667 0.78213031 0.78455606 0.79307889 0.78285494
  0.7876024  0.78088675 0.78490302 0.77933076]
 [0.82980482 0.81298086 0.84535759 0.82645839 0.81889272 0.84225504
  0.82430807 0.84220232 0.79854119 0.82052568]
 [0.80890574 0.79255027 0.8223608  0.80585153 0.79813281 0.82033024
  0.80339144 0.82181453 0.77955383 0.80053964]
 [0.81764518 0.800937   0.83246587 0.81353693 0.80752218 0.82643024
  0.81257167 0.82955542 0.78789116 0.81120866]
 [0.81089967 0.8095206  0.80810331 0.81183971 0.82069771 0.81015542
  0.81532243 0.80902158 0.81426051 0.80942118]
 [0.80399782 0.78903702 0.81784742 0.80258378 0.79549712 0.81572095
  0.80109504 0.81922771 0.77346271 0.79736773]
 [0.78011393 0.76623968 0.7936045  0.77670714 0.77163226 0.7915588
  0.77727186 